# 1. Process Files:


## [1.1] Read `configs.ini` :

In [10]:
import configparser
config = configparser.ConfigParser()

In [11]:
config.read('configs.ini')
source_dir      = config['PROCESS']['source_dir']
destination_dir = config['PROCESS']['destination_dir'] 
log_process_dir = config['PROCESS']['log_process_dir'] 
csv_process_dir = config['PROCESS']['csv_process_dir'] 

MASTER_REGEX_PHOTOS_1 = config['NOEDIT']['MASTER_REGEX_PHOTOS_1'] 
MASTER_REGEX_PHOTOS_2 = config['NOEDIT']['MASTER_REGEX_PHOTOS_2'] 
MASTER_REGEX_VIDEOS_1 = config['NOEDIT']['MASTER_REGEX_VIDEOS_1'] 
MASTER_REGEX_VIDEOS_2 = config['NOEDIT']['MASTER_REGEX_VIDEOS_2'] 


In [14]:
print("Source Directory: ",source_dir)
print("Destionation Directory: ",destination_dir)
print("Log Directory: ",log_process_dir)
print("CSV Directory: ",csv_process_dir)


Source Directory:  E:/R_PhotosVideos/R_Photos_Unsorted/Plain-11
Destionation Directory:  E:/R_PhotosVideos/R_Photos_Sorted
Log Directory:  E:/R_PhotosVideos/PythonPhotoSort/logs
CSV Directory:  E:/R_PhotosVideos/PythonPhotoSort/logs


In [15]:
print("MASTER_REGEX_PHOTOS_1 : ",MASTER_REGEX_PHOTOS_1)
print("MASTER_REGEX_PHOTOS_2 : ",MASTER_REGEX_PHOTOS_2)
print("MASTER_REGEX_VIDEOS_1 : ",MASTER_REGEX_VIDEOS_1)
print("MASTER_REGEX_VIDEOS_2 : ",MASTER_REGEX_VIDEOS_2)

MASTER_REGEX_PHOTOS_1 :  ^[iImMgG]{3}[-_]([0-9]{8})[-_].*\.(?:jpg|jpeg)$
MASTER_REGEX_PHOTOS_2 :  ^[iImMgG]{3}([0-9]{8}).*\.(?:jpg|jpeg)$
MASTER_REGEX_VIDEOS_1 :  ^[VvIiDdvideo]{3,}[-_]([0-9]{8})[-_].*\.(?:mp4)$
MASTER_REGEX_VIDEOS_2 :  ^[VvIiDd]{3}([0-9]{8}).*\.(?:mp4)$


## [1.2] Process all Photos: 

In [16]:
import re
import os
from datetime import datetime
import shutil

In [17]:
input_folder_path  = source_dir
output_folder_path = destination_dir

In [26]:
# Log file :
temp     = "log-photos-" + datetime.now().strftime("%Y%m%d-%H%M%S") + ".txt"
log_file = os.path.join(log_process_dir, temp)

# Process CSV File: 
temp     = "photos-" + datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"
csv_file = os.path.join(csv_process_dir, temp)

print("Log File: ", log_file)
print("CSV File: ", csv_file) 

Log File:  E:/R_PhotosVideos/PythonPhotoSort/logs\log-photos-20240519-160517.txt
CSV File:  E:/R_PhotosVideos/PythonPhotoSort/logs\photos-20240519-160517.csv


In [27]:
FILE_COUNT = 0
FILE_EXT_LIST = set([])
FILE_EXT_COUNTER = []

In [ ]:
csvfile_handle = open(csv_file, "w", encoding="utf-8")
csvfile_handle.write("Filename;Extension;Matched_Regex_Name;Extracted_Date;Path;Null\n")
print()

In [ ]:
src = input_folder_path
dst = output_folder_path

# OS.walk() generate the file names in a directory tree (nested subfolders) by walking the tree either top-down or bottom-up.
for root, subdirs, files in os.walk(src):
    for file in files:
        path = os.path.join(root, file)
        
        _filenameonly = file   # e.g. IMG_20150829_141244.jpg
        _extension = os.path.splitext(file)[1] # e.g. jpg

        matched_regex_name = "None"
        _extracted_ts      = "Null" # extracted timestamp 

        ## Try matching against each regex listed in config.ini

        m1 = re.search(r'{}'.format(MASTER_REGEX_PHOTOS_1), file)
        if m1: 
            _extracted_ts = m1.group(1)
            matched_regex_name = "MASTER_REGEX_PHOTOS_1"
            
        m2 = re.search(r'{}'.format(MASTER_REGEX_PHOTOS_2), file)
        if m2: 
            _extracted_ts = m2.group(1)
            matched_regex_name = "MASTER_REGEX_PHOTOS_2"

        m3 = re.search(r'{}'.format(MASTER_REGEX_VIDEOS_1), file)
        if m3: 
            _extracted_ts = m3.group(1)
            matched_regex_name = "MASTER_REGEX_VIDEOS_1"

        m4 = re.search(r'{}'.format(MASTER_REGEX_VIDEOS_2), file)
        if m4: 
            _extracted_ts = m4.group(1)
            matched_regex_name = "MASTER_REGEX_VIDEOS_2"

        
            
        _fullfilepath = path   # e.g. H:/myfolder/IMG_20150829_141244.jpg
 
        #  Filename;Extension;Matched_Regex_Name;Extracted_Date;Path;Null\n
        csv_line = _filenameonly + ";" + _extension + ";" + matched_regex_name + ";" + \
                  _extracted_ts + ";" + _fullfilepath + ";NOTHING;\n"
        
        csvfile_handle.write(csv_line)
        
        # Increment File Count
        FILE_COUNT = FILE_COUNT+ 1
        # Add file extension to set
        FILE_EXT_LIST.add(_extension)
        # Group counter
        FILE_EXT_COUNTER.append(_extension)
 

csvfile_handle.close()